In [1]:
%load_ext Cython

In [117]:
def python_insert_pop(arr0, idx_in, value_in, idx_out):
    arr = np.copy(arr0).tolist()
    if idx_in < idx_out:
        arr.insert(idx_in, value_in)
        arr.pop(idx_out+1)
    elif idx_in == idx_out:
        arr[idx_in] = value_in
    else:
        arr.insert(idx_in+1, value_in)
        arr.pop(idx_out)
    return np.array(arr, dtype=int)

In [118]:
%%cython 
import numpy as np 

def cython_insert_pop(double[:] arr, long idx_in, double value_in, long idx_out):
    cdef long n = arr.shape[0]
    cdef int i
    
    if idx_in < idx_out:
        for i in range(idx_out-1, idx_in-1, -1):
            arr[i+1] = arr[i]
    else:
        for i in range(idx_out, idx_in):
            arr[i] = arr[i+1]
    arr[idx_in] = value_in
    return np.array(arr, dtype=int)

In [121]:
n = 90
num_tests = int(1e4)

for __ in range(num_tests):
    value_in = 9
    idx_in = np.random.randint(0, n)
    idx_out = np.random.randint(0, n)
    x = np.random.randint(0, 5, n).astype('f8')
    x1 = python_insert_pop(np.copy(x), idx_in, value_in, idx_out)
    x2 = cython_insert_pop(np.copy(x), idx_in, value_in, idx_out)
    try:
        assert np.all(x1 == x2)
    except:
        np.save('x', x)
        print("Failure for x = {0}".format(x.astype(int)))
        print("idx_in, idx_out, value_in = {0}, {1}, {2}\n".format(idx_in, idx_out, value_in))
        print("python result = {0}".format(x1))
        print("cython result = {0}".format(x2))
        break

In [104]:
idx_in, idx_out

(6, 4)

In [105]:
badx = np.load('x.npy').astype(int).tolist()
print(badx)

[0, 1, 3, 4, 2, 4, 3, 2, 4]


In [106]:
idx_in, idx_out, value_in = 6, 4, 9
orig_input = [0, 1, 3, 4, 2, 4, 3, 2, 4]
corr_array = [0, 1, 3, 4, 4, 3, 9, 2, 4]

In [116]:
arr = np.copy(orig_input).tolist()
arr.insert(idx_in+1, value_in)
arr.pop(idx_out)
print(arr)
print(corr_array)

[0, 1, 3, 4, 4, 3, 9, 2, 4]
[0, 1, 3, 4, 4, 3, 9, 2, 4]


In [107]:
arr = np.copy(orig_input)
for i in range(idx_out, idx_in):
    print("Setting entry {0} from {1} to {2}".format(i, arr[i], arr[i+1]))
    arr[i] = arr[i+1]
print("\n")
print(orig_input)
print("\nReplacing entry {0} = {1} to {2}".format(idx_in, arr[i], value_in))
arr[idx_in] = value_in

print("\nresult     = {0}".format(arr.tolist()))
print("corr_array = {0}".format(corr_array))

Setting entry 4 from 2 to 4
Setting entry 5 from 4 to 3


[0, 1, 3, 4, 2, 4, 3, 2, 4]

Replacing entry 6 = 3 to 9

result     = [0, 1, 3, 4, 4, 3, 9, 2, 4]
corr_array = [0, 1, 3, 4, 4, 3, 9, 2, 4]
